# Limpieza de Tweets

In [8]:
import tweepy
import pandas as pd
import time
from datetime import datetime

# Configurar credenciales de la API
consumer_key = '6ankddKpMu4GjImcyrkGJIJ7v'
consumer_secret = 'MyllK4pyPQ175Hm0l2nbM2z5Sl1cJDVBABSRuTZEFcSShdssVS'
access_token = '3861232932-Cs250q8gQGWYVO8tTXouXspwH3njAnXfDLoo5DG'
access_token_secret = 'm1W5Xfig5MjIj4cIa9v2kA9CfyFZDiJbysiExCaF3kX2K'

# Autenticación con OAuth 1.0a
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)

# Crear instancia de API
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Lista de usuarios
usuarios = [
    'Cicmty', 'conagua_mx', 'elnortelocal', 'LoQuePasaenNL', 'eldelaconsty', 'camion_desde',
    'metrorreynlofi1', 'MovilidadSN', 'SSP_Apodaca', 'PC_NuevoLeon', 'TodosSomosHLM',
    'AtentosMTYSur', 'Apodaca_News', 'ArbSanJorge', 'SoyDeSanNicolas', 'abimaelsal',
    'BomberosNL', 'nelvaldez', 'MAGS_SP', 'saz2000', 'DarkKnightMty', 'revistacodigo21',
    'Mty_Leones', 'cesarmty', '911SanPedro', 'SSPCMonterrey', 'pc_mty', 'rayelizalder',
    'Informativo3651', 'arualsanpedrogg', 'JulioCesarCano', 'nmasmonterrey', 'info7mty',
    'ABCNoticiasMX', 'AsiEsMonterrey', 'GobSanNicolas', 'PortalElPunto',
    'gob_Escobed', 'mtygob', 'joluisgarcia', 'visionmty1', 'QuePasaEnS', 'LCAlatorre',
    'IdentidadNL', 'MtyFollow', 'ayd_monterrey', 'ComunidadMTY', 'municipiodegpe',
    'CiudaDanaMtySur'
]

# Recopilar los IDs de tweets para cada usuario
all_tweet_ids = []

for usuario in usuarios:
    try:
        # Obtener los últimos 10 tweets del usuario
        tweets = api.user_timeline(screen_name=usuario, count=10, tweet_mode='extended', include_rts=False, exclude_replies=True)

        # Procesar y guardar solo los IDs de los tweets
        for tweet in tweets:
            # Filtrar los tweets de 2024
            tweet_date = tweet.created_at
            if tweet_date.year == 2024:
                tweet_details = {
                    'Usuario': usuario,
                    'Tweet ID': tweet.id,
                    'Fecha': tweet_date
                }
                all_tweet_ids.append(tweet_details)

    except tweepy.TweepyException as e:
        print(f"Error fetching tweets for {usuario}: {e}")

    # Esperar un poco para evitar problemas de rate limit
    time.sleep(1)

# Convertir la lista de diccionarios en un DataFrame de pandas
df = pd.DataFrame(all_tweet_ids)

# Obtener la fecha actual para agregarla al nombre del archivo CSV
fecha_hoy = datetime.now().strftime('%Y-%m-%d')

# Guardar el DataFrame en un archivo CSV con la fecha de hoy en el nombre
df.to_csv(f'./Tweets_IDs_2024_{fecha_hoy}.csv', index=False)

print(f"IDs de tweets recopilados y guardados en Tweets_IDs_2024_{fecha_hoy}.csv")

Error fetching tweets for Cicmty: 401 Unauthorized
Unauthorized
Error fetching tweets for conagua_mx: 401 Unauthorized
Unauthorized
Error fetching tweets for elnortelocal: 401 Unauthorized
Unauthorized
Error fetching tweets for LoQuePasaenNL: 401 Unauthorized
Unauthorized
Error fetching tweets for eldelaconsty: 401 Unauthorized
Unauthorized
Error fetching tweets for camion_desde: 401 Unauthorized
Unauthorized
Error fetching tweets for metrorreynlofi1: 401 Unauthorized
Unauthorized
Error fetching tweets for MovilidadSN: 401 Unauthorized
Unauthorized
Error fetching tweets for SSP_Apodaca: 401 Unauthorized
Unauthorized
Error fetching tweets for PC_NuevoLeon: 401 Unauthorized
Unauthorized
Error fetching tweets for TodosSomosHLM: 401 Unauthorized
Unauthorized
Error fetching tweets for AtentosMTYSur: 401 Unauthorized
Unauthorized
Error fetching tweets for Apodaca_News: 401 Unauthorized
Unauthorized
Error fetching tweets for ArbSanJorge: 401 Unauthorized
Unauthorized
Error fetching tweets for

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Lista de palabras clave
keywords = ['lluvia', 'inundación', 'accidente', 'apagón', 'deslave', 'desborde', 'incendio', 'fuga', 'colapso', 'tormenta']

# Función para eliminar los símbolos, emoticones y links
def clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F700-\U0001F77F"  
        u"\U0001F780-\U0001F7FF"  
        u"\U0001F800-\U0001F8FF"  
        u"\U0001F900-\U0001F9FF"  
        u"\U0001FA00-\U0001FA6F"  
        u"\U0001FA70-\U0001FAFF"  
        u"\U00002700-\U000027BF"  
        u"\U0001F1E0-\U0001F1FF"  
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Función para verificar si el tweet contiene una palabra clave o sus variantes
def contains_keyword(text, keywords, nlp):
    doc = nlp(text)
    lemmas = {token.lemma_ for token in doc}
    return any(keyword in lemmas for keyword in keywords)

# Cargar la base de datos
tweets_file = "/Users/Daniel/Documents/ITESM/Séptimo Semestre/proyecto nlp/Predicci-n-de-Riesgos-Urbanos-con-NLP/X-NLP/tweets_Jun22-1.csv"
df_tweets = pd.read_csv(tweets_file)

# Eliminar tweets que no tienen texto
df_tweets_clean = df_tweets.dropna(subset=['Texto'])

# Aplicar la función de limpieza a la columna de texto 
df_tweets_clean['Texto'] = df_tweets_clean['Texto'].apply(clean_text)

# Filtrar los tweets que contienen una palabra clave o sus variantes
df_tweets_filtered = df_tweets_clean[df_tweets_clean['Texto'].apply(lambda x: contains_keyword(x, keywords, nlp))]

# Guardar el resultado filtrado
df_tweets_filtered.to_csv('/Users/Daniel/Documents/ITESM/Séptimo Semestre/proyecto nlp/Predicci-n-de-Riesgos-Urbanos-con-NLP/X-NLP/tweets_Jun_filtered/tweets_Jun22-1_filtered.csv', index=False)